# Implementing C3D Model for Video Classification

In [3]:
%load_ext autoreload
%autoreload 2

from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import pickle
import glob
import cv2
import sys
import os
import time

from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import seaborn as sns

 
# adding video-download folder to the system path
sys.path.insert(0, '/workspace/youtube-humpback-whale-classifier/video-download')
 
# importing read_frames_hdf5 function
from hdf5_data_loading import read_frames_hdf5

#ngc workspace path (where we keep our data)
workspace_path = '/mount/data'

In [2]:
import wandb

#start wandb session for metric logging
wandb.login() 

wandb.init(project="whale-classification-inception")

wandb.run.name = "c3d-training"

wandb: Currently logged in as: micheller (epg). Use `wandb login --relogin` to force relogin


In [3]:
print("Num GPUs available: ", len(tf.config.list_physical_devices('GPU'))) #1 if we select GPU mode in Colab Notebook, 0 if running on local machine

Num GPUs available:  4


## Load Dataset

In [4]:
#load dataset in
data = pd.read_csv(workspace_path + '/downloaded_videos.csv')
y = data.pop('relevant')
X = data

## Load Frames

In [5]:
#load in frames for all videos
start = time.time()

N = X.shape[0] #number of videos in our dataset
videos = np.empty((N, 30, 224, 224, 3), dtype=np.uint8)
labels = np.empty(N, dtype = np.uint8)

for i, video in enumerate(list(X.renamed_title)):
    if i % 50 == 0:
        print(f'Loading frames for video {i}...')
        
    clip_name = video.replace("_", "_clip_").replace(".mp4", "")
    frames, frame_labels = read_frames_hdf5(clip_name) #returns frames array of shape (461, 224, 224, 3)
    
    videos[i, ...] = frames[15:45] #shortened videos bc of memory issue - each video is reduced to 30 frames
    labels[i] = frame_labels[0] #all frames have the same label since label is assigned to overall video

stop = time.time()
print(f'Done loading frames in {stop-start} seconds.')
videos.shape

Loading frames for video 0...
Loading frames for video 50...
Loading frames for video 100...
Loading frames for video 150...
Loading frames for video 200...
Loading frames for video 250...
Loading frames for video 300...
Loading frames for video 350...
Done loading frames in 172.62159085273743 seconds.


(364, 30, 224, 224, 3)

## Split Data

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

train_index = list(X_train.index)
test_index = list(X_test.index)

# index data accordingly
train_videos, train_labels = videos[train_index], labels[train_index]
test_videos, test_labels = videos[test_index], labels[test_index]

# reshape label arrays as horizontal arrays
train_labels = np.reshape(train_labels, (train_labels.shape[0], 1))
test_labels = np.reshape(test_labels, (test_labels.shape[0], 1))

In [7]:
print(train_videos.shape)
print(test_videos.shape)

(291, 30, 224, 224, 3)
(73, 30, 224, 224, 3)


## Converting Data into TF Format

In [8]:
train_tf = tf.data.Dataset.from_tensor_slices((train_videos, train_labels)).batch(10)
train_tf

2022-07-25 23:37:27.760440: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-25 23:37:32.468881: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14649 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:85:00.0, compute capability: 7.0
2022-07-25 23:37:32.477385: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14649 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB-N, pci bus id: 0000:86:00.0, compute capability: 7.0
2022-07-25 23:37:32.483364: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/repli

<BatchDataset element_spec=(TensorSpec(shape=(None, 30, 224, 224, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 1), dtype=tf.uint8, name=None))>

`GPU keeps allocating 15516MiB everytime we load any sort of data, which leads to running out of memory at model training`

## Build C3D Video Classification Model

Resources
- https://towardsdatascience.com/step-by-step-implementation-3d-convolutional-neural-network-in-keras-12efbdd7b130
- https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7410867&tag=1

In [12]:
# import os
# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# print(os.getenv('TF_GPU_ALLOCATOR'))

In [4]:
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization

In [6]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

fake_data = tf.random.normal((10, 30, 224, 224, 3))
fake_label = tf.random.normal((10,))
dataset = tf.data.Dataset.from_tensor_slices((fake_data, fake_label)).batch(1)
print("Data ready")

inputs = keras.Input(shape=(30, 224, 224, 3)) #full shape w/ batch dim = (None, 30, 224, 224, 3)
x = Conv3D(64, kernel_size=(3,3,3), strides=(1,1,1))(inputs)
x = MaxPooling3D(pool_size=(2, 2, 1), strides=(1,1,1))(x)
x = Conv3D(128, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)
x = Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)
x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)
x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

x = Flatten()(x)
output = Dense(1, activation="softmax")(x)

c3d_model = keras.Model(inputs, output)
c3d_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

c3d_model.summary()

history = c3d_model.fit(dataset,
                        epochs = 16,
                        verbose= 1)

Data ready
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 30, 224, 224, 3)  0         
                             ]                                   
                                                                 
 conv3d_8 (Conv3D)           (None, 28, 222, 222, 64)  5248      
                                                                 
 max_pooling3d_5 (MaxPooling  (None, 27, 221, 222, 64)  0        
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 25, 219, 220, 128  221312    
                             )                                   
                                                                 
 max_pooling3d_6 (MaxPooling  (None, 24, 218, 219, 128  0        
 3D)                         )                  

2022-07-25 23:41:55.525667: I tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Loaded cuDNN version 8400


10/10 [==============================] - 104s 8s/step - loss: nan - accuracy: 0.0000e+00
Epoch 2/16
10/10 [==============================] - 33s 3s/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/16
10/10 [==============================] - 22s 2s/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/16
10/10 [==============================] - 22s 2s/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/16
10/10 [==============================] - 22s 2s/step - loss: nan - accuracy: 0.0000e+00
Epoch 6/16
 7/10 [====================>.........] - ETA: 6s - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [19]:
#creating C3D Model

# strategy = tf.distribute.MirroredStrategy()

# with strategy.scope():
inputs = keras.Input(shape=(30, 224, 224, 3)) #full shape w/ batch dim = (None, 30, 224, 224, 3) 

x = Conv3D(64, kernel_size=(3,3,3), strides=(1,1,1))(inputs)
x = MaxPooling3D(pool_size=(2, 2, 1), strides=(1,1,1))(x)

x = Conv3D(128, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

x = Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = Conv3D(256, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = Conv3D(512, kernel_size=(3,3,3), strides=(1,1,1))(x)
x = MaxPooling3D(pool_size=(2, 2, 2), strides=(1,1,1))(x)

x = Dense(4096, activation="relu")(x)
output = Dense(4096, activation="softmax")(x)

c3d_model = keras.Model(inputs, output)

c3d_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


history = c3d_model.fit(train_tf,
                        epochs = 16,
                        verbose= 1)

# history = c3d_model.fit(train_videos, 
#                         train_labels,
#                         validation_split = 0.2,
#                         epochs = 16,
#                         verbose= 1)

print('Done training.')

Epoch 1/16


2022-07-25 15:29:15.737747: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.97GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-07-25 15:29:32.201582: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.63GiB (rounded to 2826854400)requested by op model_1/conv3d_11/Conv3D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-07-25 15:29:32.201657: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-07-25 15:29:32.201680: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 61, Chunks in use: 60. 15.2KiB allocated for chunk

ResourceExhaustedError: Graph execution error:

Detected at node 'model_1/conv3d_11/Conv3D' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 707, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyncio/base_ev